In [1]:
%run util.ipynb
%matplotlib inline

import numpy as np
import pandas as pd
from datetime import datetime

In [66]:
# y is a numpy array
def entropy(y):
    
    # find all cases where y is equal to 1
    N = len(y)
    s1 = (y==1).sum()
    
    # cases where there is only 1 class means there is no entropy
    if s1 == 0 or N == s1:
        return 0 
    
    # Probability of y == 1
    p1 = float(s1) / N 
    p0 = 1 - p1
    
    # return the entropy 
    return -p0 * np.log2(p0) - p1 * np.log2(p1)


class TreeNode:
    
    def __init__(self, depth=0, max_depth=None):
        self.depth = depth
        self.max_depth = max_depth
        
    def fit(self, X, Y):
        
        # base case where the training data only has 1 data point or one class label
        if len(Y) == 1 or len(set(Y)) == 1:
            self.col = None
            self.split = None
            self.left = None
            self.right = None
            self.prediction = Y[0] # return whatever that class label is
            
        else:
            # Grab number of features
            D = X.shape[1]
            cols = range(D)
            
            # Need to find and store the best place to split
            max_ig = 0
            best_col = None
            best_split = None
            
            # Iterate through columns in the training data
            # find best split based on maximizing information gain
            for col in cols:
                
                # return the best split location and what the ig is
                ig, split = self.find_split(X, Y, col)
                
                # use found split is better than current max split
                if ig > max_ig:
                    max_ig = ig
                    best_col = col
                    best_split = split
            
            # base case where information gain is 0
            # take the most likely class label
            if max_ig == 0:
                self.col = None
                self.split = None
                self.left = None
                self.right = None
                self.prediction = np.round(Y.mean())
            
            # when information gain is not zero
            else:
                self.col = best_col
                self.split = best_split
                
                # case where we've hit the maximum specified depth
                # returns 2 predictions, one for the left side and one for the right side
                if self.depth == self.max_depth:
                    self.left = None
                    self.right = None
                    self.prediction = [
                        np.round(Y[X[:,best_col] < self.split].mean()),
                        np.round(Y[X[:, best_col] >= self.split].mean())
                    ]
                # use recursion otherwise to continue with the decision tree
                else:
                    # gives us the index where X is less than the best split
                    left_idx = (X[:, best_col] < best_split)
                    Xleft = X[left_idx]
                    Yleft = Y[left_idx]
                    self.left = TreeNode(self.depth + 1, self.max_depth)
                    self.left.fit(Xleft, Yleft)
                    
                    # Index where X is more than the best split
                    right_idx = (X[:, best_col] >= best_split)
                    Xright = X[right_idx]
                    Yright = Y[right_idx]
                    self.right = TreeNode(self.depth + 1, self.max_depth)
                    self.right.fit(Xright, Yright)
                
    def find_split(self, X, Y, col):
        x_values = X[:, col]
        sort_idx = np.argsort(x_values) # returns indices of sorted array
        x_values = x_values[sort_idx] # sorts the values
        y_values = Y[sort_idx] # sorted y values in exact same way
        
        # shifts y such that that compares each value against the next
        # the nonzero function returns the indices where a comparison is not the same
        boundaries = np.nonzero(y_values[:-1] != y_values[1:])
        # in summary, all it does it tell us where the boundaries are
        
        best_split = None
        max_ig = 0

        # iterate through boundaries and find the one that maxes ig
        for i in np.nditer(boundaries):
            # take midpoint of boundaries
            split = (x_values[i] + x_values[i+1]) / 2

            # calculate ig given x and y values
            ig = self.information_gain(x_values, y_values, split)
            if ig > max_ig:
                max_ig = ig
                best_split = split

        return max_ig, best_split


    def information_gain(self, x, y, split):
        # divide data based on the split
        
        y0 = y[x < split]
        y1 = y[x >= split]

        # calculate length of the data
        N = len(y)
        y0len = len(y0)

        # if only one class then no information gain
        if y0len == 0 or y0len == N:
            return 0

        # calculate the proportions
        p0 = float(len(y0)) / N
        p1 = 1 - p0

        # return the information gain
        return entropy(y) - p0*entropy(y0) - p1*entropy(y1)


    # This function predicts the class label for a particular data point
    def predict_one(self,x):

        # If a split ocurred during the fitting
        if self.col is not None and self.split is not None:
            
            # select column with highest information gain
            feature = x[self.col]

            # goes to left side
            if feature < self.split: 
                if self.left: # is there a left child?
                    p = self.left.predict_one(x) # recursive call on child
                else:
                    p = self.prediction[0] # return left prediction

            # go to the right
            else: 
                if self.right: # is there a left child?
                    p = self.right.predict_one(x) # recursive call on child
                else:
                    p = self.prediction[1] # return right prediction

        # There were no splits
        else:
            p = self.prediction

        return p

    # This function predicts the class label for all data points by repeatedly calling predict_one
    def predict(self, X):
        N = len(X)
        P = np.zeros(N)
        for i in range(N):
            P[i] = self.predict_one(X[i])
        return P

# Wrapper class for tree node, not entirely sure why this is needed other than abstraction
class DecisionTree:
    def __init__(self, max_depth = None):
        self.max_depth = max_depth
        
    def fit(self, X, Y):
        self.root = TreeNode(max_depth=self.max_depth)
        self.root.fit(X,Y)
    
    def predict(self, X):
        return self.root.predict(X)
    
    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)


In [119]:
# self written from pseudocode

# calculating entropy
def entropy(y):
    N = len(y)
    y0 = len(y[y==0])
    
    if y0 == 0 or y0 == N:
        return 0
    
    # float is critical here
    p0 = float(y0) / N
    p1 = 1 - p0
    return -p0*np.log2(p0) - p1*np.log2(p1)

class DecisionTreeNode:
    
    # set current and max depth
    def __init__(self, depth = 0, maxdepth = None):
        self.depth = depth
        self.maxdepth = maxdepth
    
    # finding possible splits in a column and which one maximizes information gain
    def find_split(self, X, Y, col):
        # sorts x and correspondingly y. Determines where y changes from one class label to another. 
        x = X[:, col]
        x_sorted_indices = np.argsort(x)
        x_sorted = x[x_sorted_indices]
        y_sorted = Y[x_sorted_indices]
        boundaries = np.nonzero(y_sorted[:-1] != y_sorted[1:])

        max_ig = 0
        best_split = None
        
        # find which boundary maximizes information gain
        for i in np.nditer(boundaries):
            split = (x[i] + x[i + 1]) / 2
            
            ig = self.information_gain(x_sorted, y_sorted, split)
            if ig > max_ig:
                max_ig = ig
                best_split = split
        
        return max_ig, best_split
            
    def information_gain(self, X, Y, split):
        # find proportion of Y less than the split and greater than the split
        
        y0 = Y[X < split]
        y1 = Y[X >= split]
        y0len = len(y0)
        N = len(Y)
        
        if y0len == 0 or y0len == N:
            return 0
        
        p0 = float(len(y0)) / N
        p1 = 1 - p0
    
        # calculate information gain
        return entropy(Y) - p0*entropy(y0) - p1*entropy(y1)
    
    def fit(self, X, Y):
        
        # if length of Y equals 1 or there is only one class left, return that class as the prediction
        if len(Y) == 1 or len(set(Y)) == 1:
            self.left = None
            self.right = None
            self.split = None
            self.col = None
            self.prediction = Y[0]
            
        # ordinary case
        else:
            D = X.shape[1]

            max_ig = 0
            best_col = None
            best_split = None
            
            # iterate through columns and find the one that maximizes information gain
            for col in range(D):

                ig, split = self.find_split(X, Y, col)
                if ig > max_ig:
                    best_col = col
                    max_ig = ig
                    best_split = split
            
            # if information gain is zero, then just take the most likely class label
            if max_ig == 0:
                self.left = None
                self.right = None
                self.split = None
                self.col = None
                self.prediction = np.round(Y.mean())
            
            # ordinary case
            else:
                self.col = best_col
                self.split = best_split
                
                # if at max depth, describe left and right predictions as most likely outcomes of the split data
                if self.depth == self.maxdepth:
                    self.left = None
                    self.right = None
                    self.prediction = [
                        np.round(Y[X[:, best_col] < self.split].mean()), 
                        np.round(Y[X[:, best_col] >= self.split].mean())
                    ]
                
                # otherwise, create left and right children using the split and recurse
                else:
                    left_idx = (X[:, best_col] < best_split)
                    Xleft = X[left_idx]
                    Yleft = Y[left_idx]
                    self.left = DecisionTreeNode(self.depth + 1, self.maxdepth)
                    self.left.fit(Xleft, Yleft)

                    right_idx = (X[:, best_col] >= best_split)
                    Xright = X[right_idx]
                    Yright = Y[right_idx]
                    self.right = DecisionTreeNode(self.depth+1, self.maxdepth)
                    self.right.fit(Xright,Yright)
    
    # loop over rows and predict each one at a time
    def predict(self, X):
        N = len(X)
        P = np.zeros(N)
        for i in range(N):
            P[i] = self.predict_one(X[i])
        return P
    
    # if there is split, recurse to the next level of the decision tree if possible; otherwise, take prediction
    # if no split, just take the prediction
    def predict_one(self, x):
        
        if self.col != None and self.split != None:
            feature = x[self.col]
            
            #go left
            if feature < self.split:
                if self.left:
                    p = self.left.predict_one(x)
                else:
                    p = self.prediction[0]
            
            #go right
            else:
                if self.right:
                    p = self.right.predict_one(x)
                else:
                    p = self.prediction[1]
        else:
            p = self.prediction
        
        return p
    
    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)

In [67]:
if __name__ == '__main__':
    X, Y = get_data()
    
    # Classifying whether number is 0 or 1
    idx = np.logical_or(Y == 0, Y ==1) # returns true where Y = 0 or 1
    # keeps indexes where idx is true
    X = X[idx] 
    Y = Y[idx]
    
    Ntrain = int(len(Y) / 2)
    Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
    Xtest, Ytest = X[Ntrain:], Y[Ntrain:]
    
    model = DecisionTree()
    model.fit(Xtrain,Ytrain)

In [68]:
print("Train accuracy:", model.score(Xtrain, Ytrain))
print("Test accuracy:", model.score(Xtest, Ytest))

Train accuracy: 1.0
Test accuracy: 0.993194192377


In [32]:
P = model.predict(Xtest)
pd.Series(P==Ytest).value_counts()

True     4384
False      24
dtype: int64

In [120]:
model2 = DecisionTreeNode()
model2.fit(Xtrain,Ytrain)

In [121]:
print("Train accuracy:", model2.score(Xtrain, Ytrain))
print("Test accuracy:", model2.score(Xtest, Ytest))

Train accuracy: 1.0
Test accuracy: 0.993421052632
